# Fine-Tuning an OpenSource Model using QLoRA

In [ ]:
%pip install -qU peft trl bitsandbytes datasets wandb

In [ ]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTConfig, SFTTrainer
from datasets import load_dataset

In [ ]:
MODEL_NAME = "mistralai/Mistral-7B-v0.1"


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


model = prepare_model_for_kbit_training(model)

In [ ]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

In [ ]:
DATA_DIR = '/content/data'

data_files = [
    f'{DATA_DIR}/all_beauty_train.parquet',
]

dataset = load_dataset('parquet', data_files=data_files, split='train')

train_test = dataset.train_test_split(train_size=100, test_size=20, seed=42)
train_dataset = train_test["train"]

test_dataset = load_dataset('parquet', data_files=[f'{DATA_DIR}/all_beauty_test.parquet'], split='train')

In [ ]:
sft_config = SFTConfig(
    output_dir="./price-prediction-qlora",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_8bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_steps=50,
    logging_steps=10,
    save_strategy="no",
    fp16=False,
    bf16=True,
    max_grad_norm=0.3,
    save_total_limit=2,
    group_by_length=True,
    report_to="none",
    packing=False,
    dataset_text_field="text",
)


trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=train_dataset,
)



In [ ]:
### Start Training

In [ ]:
print("Starting training...")

trainer.train()

trainer.model.save_pretrained("./price-prediction-final")
tokenizer.save_pretrained("./price-prediction-final")

print("Training complete! LoRA adapters saved to ./price-prediction-final")

In [ ]:

def predict_price_inmemory(prompt, model, tokenizer):

    model.eval()

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
          outputs = model.generate(
              **inputs,
              max_new_tokens=10,
              temperature=0.1,
              do_sample=False,
              pad_token_id=tokenizer.eos_token_id
          )

    result = tokenizer.decode(outputs[0], skip_special_tokens=True)

    if "Price is $" in result:
        predicted = result.split("Price is $")[-1].strip()

        import re
        match = re.search(r'(\d+\.?\d*)', predicted)
        if match:
            return match.group(1)
    return predicted

### Eye-test Validation

Not the best I know, but I wanted to go through the entire process myself and not enough time on my hands.

In [ ]:
for item in test_dataset.take(5):
  prompt = item["text"]
  actual_price = item["price"]

  predicted_price = float(predict_price_inmemory(prompt, model, tokenizer))
  print("\n" + "*" * 80)
  print(prompt)

  print(f"Prediction: ${predicted_price}.  Actual: ${actual_price}. Diff {abs(predicted_price - actual_price):,.2f}")


## Loading Somewhere in the future

It can even be loaded in a different notebook.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)


model = PeftModel.from_pretrained(base_model, "./price-prediction-final")
tokenizer = AutoTokenizer.from_pretrained("./price-prediction-final")